In [1]:
import polars as pl
import polars.selectors as cs
from utils import PathsData, import_data

In [2]:
df = import_data(PathsData.X_TEST.value)

In [3]:
df.describe()

statistic,train,gare,date,arret,p2q0,p3q0,p4q0,p0q2,p0q3,p0q4
str,str,str,str,f64,f64,f64,f64,f64,f64,f64
"""count""","""20657""","""20657""","""20657""",20657.0,20657.0,20657.0,20657.0,20657.0,20657.0,20657.0
"""null_count""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",null,null,null,18.454906,-0.184199,-0.183425,-0.094544,-0.1933,-0.188653,-0.192574
"""std""",null,null,null,6.193788,1.478776,1.441474,1.476192,1.571205,1.634196,1.723925
"""min""","""AABQSN""","""ADU""","""2023-11-13""",12.0,-32.0,-31.0,-46.0,-46.0,-66.0,-57.0
"""25%""",null,null,null,12.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
"""50%""",null,null,null,18.0,0.0,0.0,0.0,0.0,0.0,0.0
"""75%""",null,null,null,24.0,1.0,1.0,1.0,1.0,1.0,1.0
"""max""","""ZZXVUE""","""ZVY""","""2023-12-22""",36.0,6.0,6.0,9.0,11.0,6.0,9.0
